In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Step 1.1: Load Merged Data

In [3]:
# Load your merged files
clients = pd.read_csv("merged_clients_all.csv")  # Primary key is '_id'
loans = pd.read_csv("merged_loans_all.csv")      # Has 'clientID', loan_code
refunds = pd.read_csv("merged_refunds_all.csv") # Has 'clientID', 'loanID'


FileNotFoundError: [Errno 2] No such file or directory: 'merged_clients_all.csv'

## Step 1.2: Data Quality Check

In [ ]:
print("\n=== MISSING VALUES ===")
print("Clients:", clients.isnull().sum())
print("Loans:", loans.isnull().sum())
print("Refunds:", refunds.isnull().sum())

print("\n=== DATA TYPES ===")
print(loans.dtypes)
print(clients.dtypes)
loans['loanDate'] = pd.to_datetime(loans['loanDate'], dayfirst=True, errors='coerce')
print(f"Loan date range: {loans['loanDate'].min()} to {loans['loanDate'].max()}")

print(f"Duplicate loans: {loans.duplicated(subset=['clientID', 'loanDate']).sum()}")


## Step 1.3: Join Datasets

In [ ]:
# Convert clientID to integer first to remove decimal artifacts
loans['clientID'] = loans['clientID'].astype(float).astype(int).astype(str)

# Now build msid cleanly
loans['msid'] = '237' + loans['clientID']

clients['_id'] = clients['_id'].astype(str)
print(loans[['clientID', 'msid']].head())
print(clients[['_id']].head())


In [ ]:
# Make sure to create msid in loans for proper matching
#loans['msid'] = loans['clientID'].astype(str).apply(lambda x: '237' + x)

# Desired client _id
#target_client_id = '237683755851'

# Filter the loans for this client
#client_loans = loans[loans['msid'] == target_client_id]

#print(f"Number of loans for client {target_client_id}: {client_loans.shape[0]}")
#print(client_loans.head())

In [ ]:
client_loans_with_data = clients.merge(
    loans,
    left_on='_id',
    right_on='msid',
    how='inner',  # Only keeps matches; all rows will have valid loan fields!
    suffixes=('', '_loan')
)

print(f"After merge: {client_loans_with_data.shape}")
client_loans_with_data.head()
client_loans_with_data.to_csv("data/processed/client_loan_data.csv", index=False)
print("Complete client-loan_data.csv saved to: data/processed/client_loan_data.csv")

In [ ]:
# === Aggregate refunds per loan (on loanID)
refund_agg = refunds.groupby('loanID').agg({
    'amount': ['sum', 'count', 'mean', 'std'],
    'refundCapital': 'sum',
    'refundInterest': 'sum',
    'balance': 'last',
    'excess': 'sum',
    'date': ['min', 'max']
}).reset_index()
refund_agg.columns = ['loanID'] + ['refund_' + '_'.join(col).strip('_') for col in refund_agg.columns.values[1:]]
print(f"\nRefund aggregation shape: {refund_agg.shape}")
print(refund_agg.head())

In [ ]:
refund_agg.to_csv("data/processed/refund_agg.csv", index=False)

In [ ]:
# Extract the pure loan code from refund_agg['loanID'] for matching (middle part between colons):
refund_agg['loan_code2'] = refund_agg['loanID'].apply(
    lambda x: x.split(":")[1] if isinstance(x, str) and x.count(":") >= 2 else x
)

print("\nExtracted codes for matching:\n", refund_agg[['loanID', 'loan_code2']].head())

In [ ]:
client_loans_with_data['loan_code'] = client_loans_with_data['loanId'].astype(str).str.split(':').apply(lambda x: x[1] if len(x) > 1 else None)
client_loans_with_data['loan_code'].head()

In [ ]:
# === Merge refunds into loan_client using loan_code <-> loancode2
final_data = client_loans_with_data.merge(refund_agg, left_on='loan_code', right_on='loan_code2', how='inner')
print(f"\nFinal merged data shape: {final_data.shape}")
print(final_data.head())

In [ ]:
# === Save integrated file
final_data.to_csv("data/processed/integrated_data.csv", index=False)
print("Complete integrated_data.csv saved to: data/processed/integrated_data.csv")

## Step 1.4: Exploratory Data Analysis (EDA)

In [ ]:
# Basic statistics
print(final_data[['loanAmount', 'amountPaid', 'amountwithInterest']].describe())
print("\n=== Loan Status Distribution ===")
print(final_data['loanStatus'].value_counts())

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Loan amount distribution
axes[0, 0].hist(final_data['loanAmount'].dropna(), bins=50, edgecolor='black')
axes[0, 0].set_title('Loan Amount Distribution')
axes[0, 0].set_xlabel('Loan Amount')

# Interest rate distribution
axes[0, 1].hist(final_data['interest_rate'].dropna(), bins=30, edgecolor='black')
axes[0, 1].set_title('Interest Rate Distribution')
axes[0, 1].set_xlabel('Interest Rate')

# Loan status
final_data['loanStatus'].value_counts().plot(kind='bar', ax=axes[1, 1])
axes[1, 1].set_title('Loan Status Counts')
axes[1, 1].set_xlabel('Status')

plt.tight_layout()
plt.savefig('outputs/eda_distributions.png')
plt.show()

# Correlation heatmap
numeric_cols = final_data.select_dtypes(include=[np.number]).columns[:20]
plt.figure(figsize=(12, 10))
sns.heatmap(final_data[numeric_cols].corr(), annot=True, fmt='.2f', cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.tight_layout()
plt.savefig('outputs/correlation_heatmap.png')
plt.show()
